# Imports

In [7]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import pickle
from multiprocessing import Pool

import modeling
import helpers
import plotting
import scienceplots
plt.style.use(['science','no-latex'])
# plt.style.use(['science','ieee','no-latex'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

In [3]:
with open('data.pkl', 'rb') as file:
    data_dict = pickle.load(file)
    all_data = data_dict['data']
    meta_data = data_dict['meta_data']

best_params_all_backward = np.load('fitting_results/backward_params.npy')

# Parameter recovery

## V1-V1, Cluster 0, backward meta-learning model

In [ ]:
this_model = 'option_model'
structure = 'backward'
meta_learning = True
cluster = 0
experiment = 'V1-V1'
fitted_params = best_params_all_backward[(meta_data['Cluster']==cluster) & (meta_data['Experiment']==experiment)]
num_subject = fitted_params.shape[0]
niters = 1

alpha_bounds = [1e-6, 1]
beta_bounds = [1e-6, 10]
concentration_bounds = [-2, 1]
epsilon_bounds = [1e-6, 0.99]
bounds = [alpha_bounds, beta_bounds, concentration_bounds, epsilon_bounds, epsilon_bounds] 
param_names = ["alpha_2", "beta_2", 'concentration_2', 'epsilon', 'prior']

best_llh_all = np.zeros(num_subject)
best_params_all = np.zeros((num_subject, len(param_names)))

all_sim_data = {}

if __name__ == "__main__":
    inputs = []
       
    for i in range(num_subject):
        alpha_2, beta_2, concentration_2, epsilon, prior = fitted_params[i]
        sim_data = modeling.option_model(niters, 1, alpha_2, 5, beta_2, 0.2, concentration_2, epsilon, prior, experiment, structure, meta_learning=True)
        all_sim_data = helpers.concatenate_data(sim_data, all_sim_data)
        D = helpers.get_model_fit_data(sim_data, num_block=12)  
        inputs.append((this_model, D, structure, i, bounds, meta_learning))

    with Pool() as p:
        results = p.map(modeling.parallel_worker, inputs)

    for i, best_params, best_llh in results:
        best_params_all[i, :], best_llh_all[i] = best_params, best_llh

np.save(f'fitting_results/paramrec_{structure}_{experiment}_cluster{cluster}_params.npy', best_params_all)
np.save(f'fitting_results/paramrec_{structure}_{experiment}_cluster{cluster}_llh.npy', best_llh_all)


In [ ]:
plt.figure(figsize=(12,9))
for p in range(len(bounds)):
    ax = plt.subplot(2, 3, p+1)
    plt.plot(fitted_params[:,p], best_params_all[:,p], '.')
    plt.plot([bounds[p][0], bounds[p][1]], [bounds[p][0], bounds[p][1]])
    plt.xlim(bounds[p])
    plt.ylim(bounds[p])
    plt.xlabel('True')
    plt.ylabel('Recovered')
    plt.title(param_names[p])
    ax.set_box_aspect(1)

plt.suptitle(f'Parameter recovery for {this_model}, Cluster {cluster}, {experiment}')
plt.tight_layout()
plt.show()